In [2]:
import os
import numpy as np
import pandas as pd
import scipy as sp
import torch
import cell2location as c2l
from cell2location.utils.filtering import filter_genes
from cell2location.models import RegressionModel
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib as mpl
import scanpy as sc
results_folder = "/projects/b1042/Gate_Lab/projects/als-project/spatial/00.ref"

/home/zzj4347/softwares/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zzj4347/softwares/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/zzj4347/softwares/anaconda3/envs/cell2loc_env/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [3]:
all_sample_sc = sc.read_h5ad("/projects/b1042/Gate_Lab/projects/als-project/spatial/03.seurat_process/data/all_samples_seurat_01_0805.h5ad")


In [7]:
all_sample_sc.obs


,orig.ident,nCount_Spatial,nFeature_Spatial,nCount_Protein,nFeature_Protein,barcode,sample,condition,percent.mt,nCount_SCT,...,integrated_snn_res.0.1,integrated_snn_res.0.15,integrated_snn_res.0.2,integrated_snn_res.0.25,integrated_snn_res.0.3,integrated_snn_res.0.35,integrated_snn_res.0.4,integrated_snn_res.0.45,integrated_snn_res.0.5,seurat_clusters
GWF.17.24.10B.sl10___V52L18.402___B12__AACAATCCGAGTGGAC-1,SeuratProject,202.0,167,694548.0,35,AACAATCCGAGTGGAC-1,GWF.17.24.10B.sl10___V52L18.402___B12,C9orf72,2.970297,233.0,...,0,0,0,0,0,0,0,0,0,0
GWF.17.24.10B.sl10___V52L18.402___B12__AACACCATTCGCATAC-1,SeuratProject,158.0,124,358215.0,35,AACACCATTCGCATAC-1,GWF.17.24.10B.sl10___V52L18.402___B12,C9orf72,5.063291,209.0,...,0,0,0,0,0,0,0,0,0,0
GWF.17.24.10B.sl10___V52L18.402___B12__AACACGCAGATAACAA-1,SeuratProject,126.0,101,360310.0,35,AACACGCAGATAACAA-1,GWF.17.24.10B.sl10___V52L18.402___B12,C9orf72,3.968254,252.0,...,0,0,3,2,3,3,4,3,2,2
GWF.17.24.10B.sl10___V52L18.402___B12__AACACGTTGATACCGC-1,SeuratProject,455.0,379,453627.0,35,AACACGTTGATACCGC-1,GWF.17.24.10B.sl10___V52L18.402___B12,C9orf72,4.835165,430.0,...,1,2,2,3,2,2,2,2,4,4
GWF.17.24.10B.sl10___V52L18.402___B12__AACACTCGTGAGCTTC-1,SeuratProject,883.0,641,370147.0,35,AACACTCGTGAGCTTC-1,GWF.17.24.10B.sl10___V52L18.402___B12,C9orf72,2.151755,666.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MILLERMI.N14.280.CSC___V52L18.276___D7__TGTTGGTGATTAGGTA-1,SeuratProject,2077.0,1319,388565.0,35,TGTTGGTGATTAGGTA-1,MILLERMI.N14.280.CSC___V52L18.276___D7,C9orf72,5.103515,1616.0,...,0,1,1,1,1,1,1,1,1,1
MILLERMI.N14.280.CSC___V52L18.276___D7__TGTTGGTGCGCACGAG-1,SeuratProject,2393.0,1400,390830.0,35,TGTTGGTGCGCACGAG-1,MILLERMI.N14.280.CSC___V52L18.276___D7,C9orf72,5.599666,1686.0,...,2,4,6,7,7,7,7,7,7,7
MILLERMI.N14.280.CSC___V52L18.276___D7__TGTTGGTGCGCTTCGC-1,SeuratProject,693.0,458,411911.0,35,TGTTGGTGCGCTTCGC-1,MILLERMI.N14.280.CSC___V52L18.276___D7,C9orf72,7.792208,854.0,...,0,0,0,0,0,0,0,0,0,0
MILLERMI.N14.280.CSC___V52L18.276___D7__TGTTGGTGCGGAATCA-1,SeuratProject,516.0,399,511647.0,35,TGTTGGTGCGGAATCA-1,MILLERMI.N14.280.CSC___V52L18.276___D7,C9orf72,4.263566,820.0,...,0,0,0,0,7,3,4,3,7,7


In [8]:
results_folder = "/projects/b1042/Gate_Lab/projects/als-project/spatial/00.ref"
all_h5ad_files = [file for file in os.listdir(results_folder) if file.endswith("h5ad") if file != "ref.h5ad" and file != 'ref_processed.h5ad']
all_h5ad_files

['MILLERMI-AU-87-sl5___V52L19-037___D12.h5ad',
 'GWF-17-24-10B-sl10___V52L18-402___B12.h5ad',
 'GBB21-03-A12-sl5___V52L06-340___C12.h5ad',
 'GWF-18-33-10B-sl11___V52L19-028___C7.h5ad',
 'GWF_19-35_10C-sl9___V52L06-367___D2.h5ad',
 'MILLERMINEUWM_344ZLM-sl10___V52L06-367___D1.h5ad',
 'GWF-19-37-10B-sl10___V52L18-387___C5.h5ad',
 'GWF_19-47_10___V52L19-048___D3.h5ad',
 'NEULM733WR7SCC-10-sl3___V52L18-387___C6.h5ad',
 'GBB-18-01-A9-sl12___V52L06-298___C3.h5ad',
 'NEUDW867LF9_SCC_10-sl3___V52L19-016___C10.h5ad',
 'GWF-15-09-10B-sl2___V52L06-298___C4.h5ad',
 'GWF-17-24-10B-sl10___V52L19-076___B10.h5ad',
 'NEUHD481VCL_SCC_19-sl14___V52L06-340___C11.h5ad',
 'GBB20-16-A13-sl10___V52L18-332___C1.h5ad',
 'MILLERMI-N14-280-CSC___V52L18-276___D7.h5ad',
 'NEUUJ189PPH_SSC_2-5___V52L19-048___D5.h5ad',
 'GBB-17-21-B10-CNS-sl4___V52L19-037___D11.h5ad',
 'GBB19-13-A13-sl5___V52L19-016___C9.h5ad',
 'GWF-20-50-10B-sl5___V52L18-332___C2.h5ad',
 'GWF-17-23___V52L18-276___D6.h5ad']

In [ ]:

for cur_h5ad in all_h5ad_files:
    dat = sc.read_h5ad(f"{results_folder}/{cur_h5ad}")
    inf_aver = pd.read_csv(f"{results_folder}/inf_aver.csv")

    inf_aver.index = inf_aver.iloc[:]['Unnamed: 0'].astype("str")
    inf_aver.drop(columns=inf_aver.columns[0], axis=1, inplace=True)
    dat.var_names = dat.var_names.astype("str")

    c2l.models.Cell2location.setup_anndata(adata=dat)
    mod = c2l.models.Cell2location(
        dat, cell_state_df=inf_aver,
        # the expected average cell abundance: tissue-dependent
        # hyper-prior which can be estimated from paired histology:
        N_cells_per_location=7,
        # hyperparameter controlling normalisation of
        # within-experiment variation in RNA detection:
        detection_alpha=20
    )

    mod.train(
        max_epochs=15000,
        # train using full data (batch_size=None)
        batch_size=None,
        # use all data points in training because
        # we need to estimate cell abundance at all locations
        train_size=1,
        use_gpu=True,
    )
    plt.close()
    mod.plot_history(1000)
    plt.legend(labels=['full data training'])
    plt.savefig(f"/projects/b1042/Gate_Lab/projects/als-project/spatial/01.c2l_model/plots/{cur_h5ad}__ss_curve.png")
    plt.close()

    dat = mod.export_posterior(
        dat, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': True}
    )

    mod.save(f"/projects/b1042/Gate_Lab/projects/als-project/spatial/01.c2l_model/plots/{cur_h5ad}__model.pt", overwrite=True)

    sc_file = f"/projects/b1042/Gate_Lab/projects/als-project/spatial/01.c2l_model/anndata/{cur_h5ad}__after_training.h5ad"
    dat.write(sc_file)